In [4]:
import cv2
import mysql.connector
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
import pickle
from win32com.client import Dispatch

# Function to connect to the MySQL database
def connect_db():
    return mysql.connector.connect(
        host='localhost',
        user='root',
        password='Ashu@1996',
        database='myproject'
    )

# Function to fetch eye data from the database
def fetch_eye_data():
    conn = connect_db()
    cursor = conn.cursor()
    cursor.execute("SELECT adhar_number, eye FROM eyedata")
    data = cursor.fetchall()
    cursor.close()
    conn.close()
    return data

# Function to convert image bytes to numpy array
def bytes_to_image(img_bytes):
    nparr = np.frombuffer(img_bytes, np.uint8)
    img_np = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
    return img_np

# Text-to-speech function
def speak(message):
    speaker = Dispatch("SAPI.SpVoice")
    speaker.Speak(message)

# Step 1: Load the pre-trained model
with open('eye_recognition_model4.pkl', 'rb') as file:
    knn = pickle.load(file)

# Step 2: Fetch real user data from MySQL
data = fetch_eye_data()
EYES = []
LABELS = []

for adhaar_number, image in data:
    img_np = bytes_to_image(image)
    resized_img = cv2.resize(img_np, (100, 100)).flatten()  # Resize and flatten image
    EYES.append(resized_img)
    LABELS.append(adhaar_number)

# Convert lists to numpy arrays
EYES = np.array(EYES)
LABELS = np.array(LABELS)

# Optionally, you can update the model with real user data if needed
knn.fit(EYES, LABELS)

# Function to get the distance to the nearest neighbor
def get_distance_to_nearest_neighbor(knn, sample):
    distances, indices = knn.kneighbors(sample, n_neighbors=1)
    return distances[0][0]

# Set a threshold for rejecting eyes
distance_threshold = 5000  # Adjust based on your data

# Initialize video capture
video = cv2.VideoCapture(0)
eye_cascade = cv2.CascadeClassifier("c:/users/user/anaconda3/lib/site-packages/cv2/data/haarcascade_eye.xml")

already_recognized = False  # To prevent multiple recognitions

while True:
    ret, frame = video.read()  # Capture frame
    if not ret or frame is None:
        print("Failed to capture frame from camera. Exiting...")
        break  # Exit the loop if frame capture failed

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Convert frame to grayscale
    eyes = eye_cascade.detectMultiScale(gray, 1.3, 5)  # Detect eyes
    
    for (x, y, w, h) in eyes:
        crop_img = frame[y:y+h, x:x+w]  # Crop the detected eye
        resized_img = cv2.resize(crop_img, (100, 100)).flatten().reshape(1, -1)  # Resize and flatten the image
        
        if not already_recognized:  # Only recognize once
            distances, neighbors = knn.kneighbors(resized_img)
            output = knn.predict(resized_img)[0]  # Predict using KNN
            distance_to_nearest = get_distance_to_nearest_neighbor(knn, resized_img)

            # Check if the eye is recognized based on the distance threshold
            if distance_to_nearest <= distance_threshold:

                 # If recognized, show bounding box and Aadhaar number

                print(f"Your Aadhaar Number is: {output}")
                speak(str(output))
                # Draw rectangles around the recognized eye
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 1)
                cv2.rectangle(frame, (x, y), (x+w, y+h), (50, 50, 255), 2)
                cv2.rectangle(frame, (x, y-40), (x+w, y), (50, 50, 255), -1)
                cv2.putText(frame, str(output), (x, y-15), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 1)

               
            else:
                # If not recognized, speak "Not Registered" and do not show any bounding box
                speak("Not Registered")
            
            already_recognized = True  # Set flag to avoid multiple recognitions
    
    cv2.imshow('frame', frame)  # Show the video frame

    if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to quit
        break

video.release()
cv2.destroyAllWindows()


Your Aadhaar Number is: 813081439046
